<a href="https://colab.research.google.com/github/PNG-bot/NLP_WiSe_24_25/blob/MUFF_NLP_24_25/eigenes_projekt/eigenes_projekt_zeitschriften_einlesen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eigenes Projekt - Zeitschriften aus DZA einlesen

## Aus API einlesen (DZA)


In [ ]:
# Python-Bibliothek pysolr installieren
%pip install -q pysolr


### Suchindizes der Deutsche Digitale Bibliothek

Die Deutsche Digitale Bibliothek betreibt Solr-Suchindizes, die für die verschiedenen Funktionen der (Sub-) Portale benötigt werden. Das Zeitungsportal benutzt zwei Suchindizes. Eine weiterführende Dokumentation befindet sich hier: https://api.deutsche-digitale-bibliothek.de/#/search/getSolrSearch

newspaper: enthält Informationen über Zeitungstitel
Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/schema.xml
Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/solrconfig.xml

newspaper-issues: enthält die zeitungsbezogenen Metadaten inkl. Volltexte
Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/schema.xml
Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/solrconfig.xml

### Per Suchindex newspapaer abfragen


In [ ]:

import pysolr

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'location:"buenos aires" AND hasLoadedIssues:true',
    'fl': 'id,title,location,frequency,progress',
    'rows': 10  # Anzahl der zurückzugebenden Ergebnisse
}

# Suche ausführen
results = solr.search(**q)

# Ergebnisse ausgeben
for result in results:
    print(f"ID: {result.get('id', 'N/A')}")
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Location: {result.get('location', 'N/A')}")
    print(f"Frequency: {result.get('frequency', 'N/A')}")
    print(f"Progress: {result.get('progress', 'N/A')}")
    print("-" * 40)

### Per Suchindex newspaper-issue abfragen


In [ ]:
# Python-Bibliothek pandas installieren
%pip install -q pandas
%pip install -q pysolr

In [ ]:
import pandas as pd
import pysolr

solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'
solr = pysolr.Solr(solr_url, always_commit=True, timeout=10)

q = {
    'q': 'zdb_id:2149754-0 AND type:issue',
    'rows': 1000
}

response = solr.search(**q)

# Überführen der Ergebnisse in ein Pandas DataFrame
df = pd.DataFrame(response.docs)

# DataFrame anzeigen
df

### Datenanalyse
#### Puplikationszeitraum ermitteln


In [ ]:
# Sicherstellen, dass publication_date als Datumswerte formatiert sind
df['publication_date'] = pd.to_datetime(df['publication_date'])

# Frühestes und spätestes Datum ermitteln
earliest_date = df['publication_date'].min()
latest_date = df['publication_date'].max()

# Ergebnisse anzeigen
print(f"Frühestes Veröffentlichungsdatum: {earliest_date}")
print(f"Spätestes Veröffentlichungsdatum: {latest_date}")

### METS / MODS Dateien herunterladen und daraus Volltesxt Seiten generieren




In [ ]:
# Python-Bibliothek requests installieren
%pip install -q requests


In [ ]:
import os
import requests

# Verzeichnis für die XML-Dateien erstellen
directory = 'La_Otra_Alemania'
if not os.path.exists(directory):
    os.makedirs(directory)

# Durch jede Zeile des DataFrames iterieren
for index, row in df.iterrows():
    # Extrahiere den Wert der Spalte id
    item_id = row['id']

    # Formatiere den Wert der Spalte publication_date im Format YYYY-MM-DD
    publication_date = row['publication_date'].strftime('%Y-%m-%d')

    # Generiere die URL zur API-Abfrage
    url = f'https://api.deutsche-digitale-bibliothek.de/2/items/{item_id}/source/record'

    # Setze die HTTP-Header
    headers = {
        'Accept': 'application/xml'
    }

    # API-Anfrage senden
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Erstelle einen Dateipfad für die XML-Datei
        file_path = os.path.join(directory, f'{publication_date}_{item_id}.xml')

        # Speichere die XML-Datei im erstellten Verzeichnis
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Datei gespeichert: {file_path}')
    else:
        print(f'Fehler beim Abrufen der Datei für ID {item_id}: {response.status_code}')

print('Fertig!')

In [ ]:
# Python-Bibliothek lxml installieren
%pip install -q lxml

In [ ]:
rom lxml import etree

# Definiere die Namensräume
NAMESPACES = {
    'mets': 'http://www.loc.gov/METS/',
    'xlink': 'http://www.w3.org/1999/xlink'
}

# Verzeichnisse definieren
xml_directory = 'La_Otra_Alemania'
download_directory = 'La_Otra_Alemania'

# Erstelle das Download-Verzeichnis, falls es nicht existiert
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Funktion, um URLs aus einer XML-Datei zu extrahieren und herunterzuladen
def download_files_from_xml(xml_file_path, xpath_expr, subfolder, extension):
    # Lade die XML-Datei ein
    with open(xml_file_path, 'rb') as xml_file:
        tree = etree.parse(xml_file)

    # Extrahiere die URLs mit dem gegebenen XPath-Ausdruck
    urls = tree.xpath(xpath_expr, namespaces=NAMESPACES)

    # Erstelle das Unterverzeichnis, benannt nach der XML-Datei
    xml_file_name = os.path.splitext(os.path.basename(xml_file_path))[0]
    destination_dir = os.path.join(download_directory, xml_file_name, subfolder)
    os.makedirs(destination_dir, exist_ok=True)

    # Lade jede URL herunter und speichere die Datei
    for i, url in enumerate(urls, start=1):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Überprüfe auf HTTP-Fehler
            file_path = os.path.join(destination_dir, f'{i}.{extension}')
            with open(file_path, 'wb') as output_file:
                output_file.write(response.content)
            print(f'Downloaded {url} to {file_path}')
        except requests.RequestException as e:
            print(f'Failed to download {url}: {e}')

# Durchlaufe alle XML-Dateien im Verzeichnis
for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_file_path = os.path.join(xml_directory, xml_file)
        # Lade und speichere Dateien mit dem ersten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href', 'DEFAULT', 'jpeg')
        # Lade und speichere Dateien mit dem zweiten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href', 'DDB_FULLTEXT', 'xml')

## Alto-XML-Sateien einlesen

### Vorbereitung


In [ ]:
import pandas as pd
from lxml import etree
from tqdm import tqdm
import unicodedata
from bs4 import BeautifulSoup
from pathlib import Path

Optional: Lokale Installation


In [ ]:
%pip install -q tqdm
%pip install -q bs4
%pip install -q pathlib
%pip install -q matplotlib
%pip install -q numpy
%pip install -q wordcloud
%pip install -q plotly.express
%pip install -q spacy
%pip install -q nltk

### Exemplarisches Laden in EINER XML-Datei

Als einfache Datei laden

In [ ]:
with open ("La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml") as f:
    content = f.read()
print(content)
print(type(content))

Direkt als XML laden

In [ ]:
xml = BeautifulSoup(open('La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4/DDB_FULLTEXT/5.xml'),'lxml-xml')
print(xml.prettify())
print(type(xml))

### Laden aller Dateien aus einem Unterordner

In [ ]:
folder_xml = []

for filepath in Path('./La_Otra_Alemania/1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4').glob('*/*.XML'):
    with filepath.open() as f:
        soup = BeautifulSoup(f,'lxml-xml')
        folder_xml.append(soup)
print(len(folder_xml))
print(folder_xml)

### Laden ALLER .xml-Dateien im Unterverzeichnis

In [ ]:
all_xml = {}

for filepath in Path('./La_Otra_Alemania').glob('**/*.XML'):
    with filepath.open(encoding='utf-8') as f:
        # read as string:
        xml_string = f.read()
        #Dateiname inklusive der Namen der beiden übergeordneten Ordner als Schlüssel:
        key = f"{filepath.parent.parent.name}/{filepath.parent.name}/{filepath.name}"
        all_xml[key] = xml_string
print(len(all_xml))

#### Kurze Erkärung zu .glob:

- Mustererkennung: Die glob-Methode wird verwendet, um Dateipfade mit einem bestimmten Muster abzugleichen. In diesem Fall ist das Muster '\*/\*.XML'.
- Musterdetails:
    - '\*' entspricht einer beliebigen Anzahl von Zeichen, einschließlich keinem.
    - '\*.XML' entspricht jeder Datei mit der Erweiterung '.XML'.
    - Das Muster '\*/\*.XML' sucht speziell nach '.XML'-Dateien, die sich eine Verzeichnisebene unterhalb des angegebenen Pfads ('./Folder') befinden.
- Rekursive Suche: Das Muster '\*/\*.XML' sucht nach XML-Dateien in allen unmittelbaren Unterverzeichnissen von './Folder'. Um rekursiv durch alle Unterverzeichnisse in beliebiger Tiefe zu suchen, wird das Muster '\*\*/\*.XML' verwendet.

#### Umwandlung in ein Pandas-Dataframe

In [ ]:
df = pd.DataFrame(list(all_xml.items()), columns=['filename', 'content'])
df

### Suchen nach möglichen Problemen (optional)

In [ ]:
# Sucht nach dem Index von Reihen, in denen die Spalte "text" keinen Inhalt hat:
empty_text_indices = df[df['text'] == ""].index

# Ausgabe der gefundenen Zeilen ohne Text:
for index in empty_text_indices:
    print(f"Index: {index}, Content: {df.loc[index, 'Filename']}")

### Text aus XML extrahieren


In [ ]:
def extract_text(content):
    # Remove XML declaration if present
    if content.startswith('<?xml'):
        content = content.split('?>', 1)[1]

    NS = {'alto': 'http://www.loc.gov/standards/alto/ns-v2#'}
    tree = etree.fromstring(unicodedata.normalize("NFC", content))

    text_lines = []  # Initialize as an empty list to store text lines

    for line in tree.xpath('//alto:TextLine', namespaces=NS):
        text = " ".join(
            word for word in line.xpath('alto:String/@CONTENT', namespaces=NS))
        text_lines.append(text)  # Append each extracted text line to the list

    return " ".join(text_lines)  # Return all text as a single string

Erstellen einer neuen Spalte "text". In diese wird das jeweilige Ergebnis der Anwendung der Funktion "extract_text" auf das korrespondierende Element in der Spalte "content" des Dataframes "df" geschrieben:

In [ ]:
df['text'] = df['content'].apply(extract_text)
df

Löschen der Spalte "content" um Speicherplatz zu sparen:

In [ ]:
df2 = df.drop(columns=['content'])
df2

Speichern als CSV:


In [ ]:
df2.to_csv("otra_alemania_content.csv", encoding = "UTF-8")